<a href="https://colab.research.google.com/github/pablojrios/fluence_maps/blob/master/tf2_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compute predictions on a TF dataset using an stored .h5 model

In [1]:
#%cd /content/drive/My\ Drive/Healthcare/Radioterapia/data/ciolaplata
#!unrar x /content/drive/My\ Drive/Healthcare/Radioterapia/Mapas\ CIO\ La\ Plata/Mapas\ Calculados/2019.rar
#!ls -l 2015/*dcm | wc -l

import os

%cd -q '/content'
if os.path.exists('fluence_maps'):
  !rm -fr fluence_maps

GIT_USERNAME = "pablojrios"
GIT_TOKEN = "1d88a0b85d2b00a03796e4d8b7e5f7b249b12f9b"
!git clone -s https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/fluence_maps.git

Cloning into 'fluence_maps'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 243 (delta 138), reused 95 (delta 39), pack-reused 0
Receiving objects: 100% (243/243), 1.54 MiB | 1.74 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [2]:
import numpy as np
import tensorflow as tf
%cd '/content/fluence_maps'
from util.dataset import _tfrecord_dataset_type_from_folder, _parse_jpeg_image_function
from util.preprocess import rescale_0_to_1
import os
import pandas as pd

/content/fluence_maps


In [3]:
print('Tensorflow version = {}'.format(tf.__version__))
print('Executing eagerly = {}'.format(tf.executing_eagerly()))

Tensorflow version = 2.4.0
Executing eagerly = True


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls -l '/content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata/models'

total 5628284
-rw------- 1 root root 204753152 May  6  2020 1588803713.h5
-rw------- 1 root root 204752992 May  8  2020 1588961914.h5
-rw------- 1 root root 204752992 Jun  8  2020 1591581915.h5
-rw------- 1 root root 204753488 Jan  3 19:04 1609700652.h5
-rw------- 1 root root 233090768 Jan  3 19:22 1609701766.h5
-rw------- 1 root root 119785520 Jan  3 19:30 1609702231.h5
-rw------- 1 root root 119785520 Jan  3 19:39 1609702768.h5
-rw------- 1 root root  52148296 Jan  3 20:25 1609705507.h5
-rw------- 1 root root  55218296 Jan  3 20:36 1609706214.h5
-rw------- 1 root root  44420192 Jan  3 20:49 1609706993.h5
-rw------- 1 root root 204753488 Jan  3 21:04 1609707882.h5
-rw------- 1 root root 156500328 Jan  3 21:16 1609708579.h5
-rw------- 1 root root 204753488 Jan  3 21:32 1609709567.h5
-rw------- 1 root root 204754632 Jan  3 21:45 1609710337.h5
-rw------- 1 root root 204754632 Jan  3 21:54 1609710864.h5
-rw------- 1 root root 204754632 Jan  3 22:01 1609711277.h5
-rw------- 1 root root 204

In [6]:
#============================DEFINE YOUR ARGUMENTS==============================
# base data directory
ARG_DATASET_DIR='/content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata'
# folder under ARG_DATASET_DIR path.
ARG_TFDATASET_FOLDER='tfds.2019.localnorm'
ARG_MODEL_NAME = '1610755621'
# 'train', 'validation', 'test' 
ARG_PART = 'validation'
ARG_TRANSFORM_GAMMA=True

In [7]:
tfdataset_dir = os.path.join(ARG_DATASET_DIR, ARG_TFDATASET_FOLDER)
raw_test = _tfrecord_dataset_type_from_folder(tfdataset_dir, ARG_PART)
print(raw_test)

<TFRecordDatasetV2 shapes: (), types: tf.string>


In [8]:
# Apply this function to each item in the dataset using the map method:
num_workers = 8
IMG_SIZE = 256
normalization_fn = rescale_0_to_1
test = raw_test.map(lambda e: _parse_jpeg_image_function(e, IMG_SIZE, normalization_fn, transform_gamma=ARG_TRANSFORM_GAMMA), num_parallel_calls=num_workers)
print(test)

<ParallelMapDataset shapes: ((256, 256, 3), (), ()), types: (tf.float32, tf.float32, tf.string)>


In [9]:
gamma_values = test.map(lambda image, gamma, filename: gamma)
gamma_values = np.array(list(gamma_values.as_numpy_iterator()))
BATCH_SIZE = 32 # mae puede variar según batch size.
test_batches = test.batch(BATCH_SIZE)
print(test_batches)

<BatchDataset shapes: ((None, 256, 256, 3), (None,), (None,)), types: (tf.float32, tf.float32, tf.string)>


In [10]:
gamma_values

array([10.231887 ,  9.359355 , 10.358353 ,  9.396875 , 10.318111 ,
       11.890144 , 10.67689  , 11.203389 , 11.547211 , 11.894532 ,
       11.193885 ,  8.830025 , 11.38552  ,  1.9224298,  3.173596 ,
        6.142882 ,  8.985668 , 10.51779  , 11.218861 , 11.854728 ,
        8.34655  , 11.181201 ,  6.118203 , 11.588353 , 11.383839 ,
        7.9336762, 10.152645 , 11.304118 , 10.392793 ,  6.259453 ,
        9.472984 , 11.885437 , 10.893768 ,  9.058377 ,  8.504367 ,
       10.084724 , 10.718363 ,  8.79546  , 10.416525 ,  9.372651 ,
       11.73886  ,  6.1671944, 11.88912  ,  8.744701 ,  8.018712 ,
       11.829034 ,  2.2850528, 11.301259 ,  6.1518292, 10.401095 ,
       10.878398 ,  5.229715 , 10.895308 ,  8.59919  ,  5.7354813,
        9.531827 , 11.2401705,  9.511285 ,  9.25397  , 11.582056 ,
        9.452694 ,  9.292822 ,  9.466713 ,  6.4773827, 11.689602 ,
        3.3312604,  8.072112 ,  8.598578 , 11.723233 , 11.251089 ,
        9.773263 , 10.742828 ,  5.2675004, 10.260441 , 10.6381

In [11]:
# load model
dir = os.path.join(ARG_DATASET_DIR, "models")
saved_model_dir = '{}/{}.h5'.format(dir, ARG_MODEL_NAME)
print(f'Loading model {saved_model_dir}...')
loaded_model = tf.keras.models.load_model(saved_model_dir)

Loading model /content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata/models/1610755621.h5...


In [12]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 12,062,209
Non-trainable params: 2,915,648
_________________________________________________________________


In [19]:
# Evaluate dataset with the loaded model to calculate loss (mae) because
# metric value could differ from the one reported during training.
tmp_test_batches = test_batches.map(lambda image, gamma, filename: (image, gamma))
print(tmp_test_batches)
loss, mse = loaded_model.evaluate(tmp_test_batches, workers=num_workers, verbose=0)
print('\n\nLoaded model, test loss: {:5.4f}'.format(loss))
print('Loaded model, test mse: {:5.4f}'.format(mse))

<MapDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.float32)>


Loaded model, test loss: 1.2626
Loaded model, test mse: 3.2588


In [42]:
inputs=loaded_model.get_layer("vgg16").input
outputs=loaded_model.get_layer("vgg16").output
cnn_part=tf.keras.models.Model(inputs, outputs)
cnn_part.summary()
predictions = cnn_part.predict(tmp_test_batches)

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0  

TypeError: ignored

In [56]:
predictions[0][:,:,3]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.04714972],
       [0.42540392, 0.52526826, 0.02264738, 0.        , 0.        ,
        0.20159179, 0.52504116, 0.40518296],
       [0.5523675 , 0.6336503 , 0.        , 0.        , 0.        ,
        0.        , 0.6909603 , 0.5508331 ],
       [0.55383277, 0.65856797, 0.10184079, 0.        , 0.        ,
        0.        , 0.7007607 , 0.5546411 ],
       [0.55295634, 0.7224965 , 0.6035311 , 0.        , 0.        ,
        0.49658456, 0.7226466 , 0.5548006 ],
       [0.55215436, 0.7544489 , 0.7779123 , 0.6199256 , 0.5343156 ,
        0.75826055, 0.7535164 , 0.5501704 ],
       [0.5623391 , 0.7399477 , 0.7790106 , 0.76301706, 0.7483633 ,
        0.76404566, 0.7365285 , 0.5516058 ],
       [0.457116  , 0.57710373, 0.611019  , 0.59604764, 0.59551644,
        0.5958221 , 0.5283896 , 0.36009964]], dtype=float32)

In [47]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(predictions)
feature_batch_average[0]

<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([0.0000000e+00, 0.0000000e+00, 3.2391604e-03, 4.0733489e-01,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.8115321e-03, 0.0000000e+00, 0.0000000e+00, 7.0695907e-02,
       0.0000000e+00, 7.1265101e-02, 3.3663481e-01, 0.0000000e+00,
       0.0000000e+00, 4.7827575e-01, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 2.3724651e-04, 1.8751528e-04,
       2.7491548e-04, 1.3349913e-03, 6.5133674e-04, 0.0000000e+00,
       4.5897409e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 2.3647550e-01, 0.0000000e+00,
       0.0000000e+00, 1.3189678e-01, 0.0000000e+00, 2.2064770e-02,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 7.6249218e-01,
       6.28917

In [75]:
from math import isclose
a=predictions[0][:,:,3].mean()
b=feature_batch_average[0][3].numpy()
isclose(a, b, rel_tol=1e-7)

True

In [ ]:
# Make predictions
predictions = loaded_model.predict(tmp_test_batches)
# from (1121,1) to (1121,); ie.: ndim = 2 to ndim = 1
predictions = predictions.reshape(-1)

In [ ]:
tmp_test_batches = test.map(lambda image, gamma, filename: (filename, gamma))
lst = [(filename.numpy().decode('utf-8'), gamma.numpy()) for filename, gamma in tmp_test_batches]
lst2 = [(e[0], e[1], p) for e, p in zip(lst, predictions)]

# armar un pandas dataframe con el test set completo
df = pd.DataFrame(lst2, columns=['filename', 'actual gamma', 'predicted gamma'])
dir = os.path.join(ARG_DATASET_DIR, "predictions")
predictions_file_path = '{}/predicted_gamma_{}.{}.csv'.format(dir, ARG_MODEL_NAME, ARG_PART)
df.to_csv(predictions_file_path, index=False)
print(f'Predictions saved to {predictions_file_path}.')

Predictions saved to /content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata/predictions/predicted_gamma_1610755621.validation.csv.


In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
